# Playbook Explorer

In [1]:
import os
from pathlib import Path

import networkx as nx
import matplotlib.pyplot as plt

In [2]:
import pymbe.api as pm

from pymbe.client import SysML2Client
from pymbe.interpretation.calc_dependencies import generate_execution_order
from pymbe.interpretation.interp_playbooks import *
from pymbe.interpretation.results import *
from pymbe.graph.lpg import SysML2LabeledPropertyGraph
from pymbe.label import get_label
from pymbe.query.metamodel_navigator import map_inputs_to_results

## Key IDs

In [3]:
fts_full_mass = '004a1b5f-4bfc-4460-9f38-1e7b4caba6e5'
ft200_full_mass = '1e5a0ed7-8b41-4ab4-a433-8f7eedd75833'
ft100_full_mass = 'a57b423b-5c0c-4057-be6b-689abcb536b2'
liquid_stage_full_mass = '7beafac8-c1c1-4b1b-ae21-d3c9a733531c'
top_plus = 'b51bb349-e210-4be8-be64-e749ea4e563b'
tank_mass_sum_1 = '700d97d1-410a-459c-ad09-8792c27e2803'
collect_1 = 'd6644a0a-6eef-49c1-a770-60886073554c'
collect_1_result = '31f8c4bd-9700-4bc3-9970-3eb5451f0203'
full_mass_dot = 'ad0bff53-eebe-4446-a8df-4db0b7187707'
fre_1 = '2665fb1b-1f12-4f13-a977-0f060915773e'
fre_1_result = '6cfb516b-6045-454e-a521-83b747acef7e'

In [4]:
helper_client = SysML2Client()

elements_data_path = Path("..") / "tests" / "data" / "Kerbal" / "elements.json"
assert elements_data_path.exists(), f"Could not find: '{elements_data_path}'"

helper_client._load_from_file(elements_data_path)

In [5]:
lpg = SysML2LabeledPropertyGraph()
lpg.update(helper_client.elements_by_id, False)

In [6]:
shorten_pre_bake = {
    'RT-10 "Hammer" Solid Fuel Booster': "RT-10",
    'RT-5 "Flea" Solid Fuel Booster': "RT-5",
    'LV-T45 "Swivel" Liquid Fuel Engine': "LV-T45",
    'FL-T100 Fuel Tank': "FL-T100",
    'FL-T200 Fuel Tank': "FL-T200"
}

In [7]:
m0_interpretation = random_generator_playbook(
    helper_client,
    lpg,
    shorten_pre_bake,
)

In [8]:
feat_sequences = build_sequence_templates(lpg=lpg)

In [9]:
from pymbe.query.query import roll_up_upper_multiplicity, roll_up_multiplicity_for_type

total = 0
for seq in feat_sequences:
    if lpg.nodes[seq[-1]]['type'][0]['@id'] == 'ede2b2e7-9280-4932-9453-134bf460892f':
        total = total + roll_up_upper_multiplicity(lpg, lpg.nodes[seq[-1]])
        print(str(pprint_single_id_list(seq, lpg.nodes)) + ", " + str(roll_up_upper_multiplicity(lpg, lpg.nodes[seq[-1]])))
print(total)

print(roll_up_multiplicity_for_type(lpg, lpg.nodes["ede2b2e7-9280-4932-9453-134bf460892f"], "upper"))

['RT-10 "Hammer" Solid Fuel Booster', 'Full Mass: Real'], 40
['RT-10 "Hammer" Solid Fuel Booster', 'Empty Mass: Real'], 40
['RT-10 "Hammer" Solid Fuel Booster', 'Specific Impulse: Real'], 40
['RT-10 "Hammer" Solid Fuel Booster', 'Thrust: Real'], 40
['Liquid Stage', 'Full Mass: Real'], 5
['Liquid Stage', 'Empty Mass: Real'], 5
['Rocket', 'stages: Rocket Stage', 'Payload Mass: Real'], 5
['Rocket', 'stages: Rocket Stage', 'Loaded Mass: Real'], 5
['Rocket', 'stages: Rocket Stage', 'Burnout Mass: Real'], 5
['Rocket', 'stages: Rocket Stage', 'Coupler to Carrying Stage: Coupler', 'Separation Force: Real'], 40
['Solid Stage', 'Full Mass: Real'], 5
['Solid Stage', 'Empty Mass: Real'], 5
['Fuel Tank Section', 'Oxidizer: Real'], 150
['Fuel Tank Section', 'Full Mass: Real'], 150
['Fuel Tank Section', 'Empty Mass: Real'], 150
['Fuel Tank Section', 'Liquid Fuel: Real'], 150
['Liquid Engine', 'Thrust: Real'], 40
['Liquid Engine', 'Specific Impulse: Real'], 40
['Kerbal Rocket Part', 'Max Temperature: 

In [10]:
dcg = generate_execution_order(lpg, m0_interpretation)

/home/santiago/sandboxes/pymbe/src/pymbe/graph/lpg.py:368: UserWarning: These edge types are not in the graph: {'ImpliedParameterFeedforward'}.
  warn(f"These edge types are not in the graph: {mismatched_edge_types}.")


In [11]:
from pymbe.graph.calc_lpg import CalculationGroup
cg = CalculationGroup(lpg.get_projection("Expression Inferred Graph"), m0_interpretation, dcg)

In [12]:
for execution_step in dcg:
    if lpg.nodes[execution_step[0]]["@type"] == "OperatorExpression":
        print("=========")
        print(get_label_for_id(execution_step[0], lpg.nodes))
        print(get_label_for_id(execution_step[1], lpg.nodes))

collect ($collection) => $result
$result
collect ($collection) => $result
$result
+ ($x, $y) => $result
$result


In [13]:
for item in dcg:
    print(str(pprint_single_id_list(item[0:2], lpg.nodes)) + ", " + item[2])

['140.0 «Occurred LiteralReal»', '$result'], Output
['170.0 «Occurred LiteralReal»', '$result'], Output
['$result', 'Specific Impulse: Real'], ValueBinding
['$result', 'Specific Impulse: Real'], ValueBinding
['Specific Impulse: Real', 'Specific Impulse: Real'], Assignment
['Specific Impulse: Real', 'Specific Impulse: Real'], Assignment
['1 «Occurred LiteralInteger»', '$result'], Output
['3.56 «Occurred LiteralReal»', '$result'], Output
['1.5 «Occurred LiteralReal»', '$result'], Output
['$result', 'Full Mass: Real'], ValueBinding
['$result', 'Full Mass: Real'], ValueBinding
['Full Mass: Real', 'Full Mass: Real'], Assignment
['Full Mass: Real', 'Full Mass: Real'], Assignment
['1 «Occurred LiteralInteger»', '$result'], Output
['8 «Occurred LiteralInteger»', '$result'], Output
['0 «Occurred LiteralInteger»', '$result'], Output
['167.97 «Occurred LiteralReal»', '$result'], Output
['$result', 'Thrust: Real'], ValueBinding
['Thrust: Real', 'Thrust: Real'], Assignment
['197.9 «Occurred Literal

In [14]:
cg.solve_graph(lpg)

Calling collect with base = LS#0, collection input LS#0.+ ($x, $y) => $result.sum ($collection) => $result.collect ($collection) => $result.$collection#0 ([[LS#0, FL-T200#22], [LS#0, FL-T100#14], [LS#0, FL-T200#89], [LS#0, FL-T200#34], [LS#0, FL-T200#96], [LS#0, FL-T200#63], [LS#0, FL-T200#92], [LS#0, FL-T100#7], [LS#0, FL-T200#76], [LS#0, FL-T100#17], [LS#0, FL-T100#27], [LS#0, FL-T200#17], [LS#0, FL-T200#16], [LS#0, FL-T200#104], [LS#0, FL-T200#1], [LS#0, FL-T200#40], [LS#0, FL-T200#9], [LS#1, FL-T200#105], [LS#1, FL-T200#90], [LS#1, FL-T200#42], [LS#1, FL-T100#37], [LS#1, FL-T100#11], [LS#1, FL-T100#20], [LS#1, FL-T100#3], [LS#1, FL-T200#69], [LS#1, FL-T200#4], [LS#1, FL-T200#64], [LS#1, FL-T100#32], [LS#1, FL-T200#101], [LS#1, FL-T200#18], [LS#1, FL-T200#83], [LS#1, FL-T100#13], [LS#1, FL-T200#49], [LS#1, FL-T200#70], [LS#1, FL-T200#15], [LS#1, FL-T200#43], [LS#1, FL-T100#15], [LS#1, FL-T200#94], [LS#1, FL-T200#74], [LS#1, FL-T100#24], [LS#1, FL-T200#107], [LS#1, FL-T200#93], [LS#2

In [15]:
for print_line in pprint_interpretation(m0_interpretation, lpg.nodes):
    print(print_line)

Real, id = ede2b2e7-9280-4932-9453-134bf460892f, size = 3010
[.Real#0 (unset)]
[.Real#1 (unset)]
[.Real#2 (0.75)]
[.Real#3 (unset)]
[.Real#4 (0.125)]
['..']
Coupler, id = 9e6e4ffc-1ad9-4351-bfb6-882de5c73c74, size = 40
[Coupler#0]
[Coupler#1]
[Coupler#2]
[Coupler#3]
[Coupler#4]
['..']
RT-10 "Hammer" Solid Fuel Booster, id = 8851ab1c-0d7f-4fe2-bee0-8b29d408c897, size = 18
[RT-10#0]
[RT-10#1]
[RT-10#2]
[RT-10#3]
[RT-10#4]
['..']
RT-5 "Flea" Solid Fuel Booster, id = 5be56a39-f4a4-4fbb-872c-12f3e717593c, size = 22
[RT-5#0]
[RT-5#1]
[RT-5#2]
[RT-5#3]
[RT-5#4]
['..']
Solid Stage, id = b473978d-40de-4809-acef-4793f738c44e, size = 1
[SS#0]
Liquid Stage, id = e6c22f19-e5e0-4a4b-9a3f-af2f01382465, size = 4
[LS#0]
[LS#1]
[LS#2]
[LS#3]
FL-T200 Fuel Tank, id = cc585eec-c66c-48aa-b319-1395a0c8e292, size = 109
[FL-T200#0]
[FL-T200#1]
[FL-T200#2]
[FL-T200#3]
[FL-T200#4]
['..']
FL-T100 Fuel Tank, id = 1eca9960-e445-4d2f-be3f-cd7a6882435d, size = 41
[FL-T100#0]
[FL-T100#1]
[FL-T100#2]
[FL-T100#3]
[FL-T1